In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 5GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/MLChallenge-2/Test.csv
/kaggle/input/MLChallenge-2/final.csv


In [2]:
df = pd.read_csv('/kaggle/input/MLChallenge-2/final.csv')
df_test = pd.read_csv('/kaggle/input/MLChallenge-2/Test.csv')

In [3]:
df_test.shape

(50, 14)

In [4]:
df.shape
df.head(10)

,ID,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,29,65,0,2,140,417,1,0,157,0,0.8,2,1,2,1
1,57,48,1,0,122,222,0,0,186,0,0.0,2,0,2,1
2,9,52,1,2,172,199,1,1,162,0,0.5,2,0,3,1
3,147,44,0,2,118,242,0,1,149,0,0.3,1,1,2,1
4,303,57,0,1,130,236,0,0,174,0,0.0,1,1,2,0
5,65,58,1,2,140,211,1,0,165,0,0.0,2,0,2,1
6,179,43,1,0,120,177,0,0,120,1,2.5,1,0,3,0
7,243,64,1,0,145,212,0,0,132,0,2.0,1,2,1,0
8,126,34,0,1,118,210,0,1,192,0,0.7,2,0,2,1
9,94,54,0,1,132,288,1,0,159,1,0.0,2,1,2,1


In [5]:
df['target'] = df.target.map({0: 0, 1: 1, 2: 1, 3: 1, 4: 1})
df['sex'] = df.sex.map({0: 'female', 1: 'male'})
df['thal'] = df.thal.fillna(df.thal.mean())
df['ca'] = df.ca.fillna(df.ca.mean())


In [6]:
df_test['target'] = df.target.fillna(df.target.mean())
df_test.head(10)

,ID,age,sex,cp,trestbps,chol,fbs,restecg,thalach,exang,oldpeak,slope,ca,thal,target
0,128,67,0,2,152,277,0,1,172,0,0.0,2,1,2,1
1,232,57,1,0,165,289,1,0,124,0,1.0,1,3,3,1
2,220,48,1,0,130,256,1,0,150,1,0.0,2,2,3,1
3,184,58,1,2,112,230,0,0,165,0,2.5,1,1,3,1
4,25,40,1,3,140,199,0,1,178,1,1.4,2,0,3,0
5,56,52,1,1,134,201,0,1,158,0,0.8,2,1,2,1
6,28,51,1,2,110,175,0,1,123,0,0.6,2,0,2,0
7,125,39,0,2,94,199,0,1,179,0,0.0,2,0,2,0
8,139,57,1,0,110,201,0,1,126,1,1.5,1,0,1,1
9,67,51,1,2,100,222,0,1,143,1,1.2,1,0,2,1


In [7]:
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier

In [8]:
cat_list = ['sex', 'cp', 'fbs', 'restecg', 'exang', 'slope', 'ca', 'thal']

In [9]:
for ele in cat_list:
    add = pd.get_dummies(df[ele], prefix=ele)
    df1 = df.join(add)# join columns with old dataframe
    df = df1
print (df.head(3))
print (df.info())

   ID  age     sex  cp  trestbps  chol  fbs  restecg  thalach  exang  ...  \
0  29   65  female   2       140   417    1        0      157      0  ...   
1  57   48    male   0       122   222    0        0      186      0  ...   
2   9   52    male   2       172   199    1        1      162      0  ...   

   slope_2  ca_0  ca_1  ca_2  ca_3  ca_4  thal_0  thal_1  thal_2  thal_3  
0        1     0     1     0     0     0       0       0       1       0  
1        1     1     0     0     0     0       0       0       1       0  
2        1     1     0     0     0     0       0       0       0       1  

[3 rows x 40 columns]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 253 entries, 0 to 252
Data columns (total 40 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   ID          253 non-null    int64  
 1   age         253 non-null    int64  
 2   sex         253 non-null    object 
 3   cp          253 non-null    int64  
 4   trestbps    

In [10]:
x_train = df[["oldpeak","age"]]
y_train = df[["target"]]

In [11]:
x_test = df_test[["oldpeak","age"]]
y_test = df_test[["target"]]

 Using Random Forest Classifier

In [14]:
from sklearn.ensemble import RandomForestClassifier

In [15]:
rf_scores = []
estimators = [10, 100, 200, 500, 1000]
for i in estimators:
    rf_classifier = RandomForestClassifier(n_estimators = i, random_state = 0)
    rf_classifier.fit(x_train, y_train)
    rf_scores.append(rf_classifier.score(x_test, y_test))

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  """
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:5: DataConversionWarning: A column-vect

In [16]:
print("The score for Random Forest Classifier is {}% with {} estimators.".format(rf_scores[1]*100, [100, 500]))

The score for Random Forest Classifier is 48.0% with [100, 500] estimators.


In [22]:
from sklearn.ensemble import RandomForestClassifier
classifier = RandomForestClassifier(n_estimators = 10)
classifier.fit(x_train, y_train)


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:3: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  This is separate from the ipykernel package so we can avoid doing imports until


RandomForestClassifier(n_estimators=10)

In [23]:
y_pred = classifier.predict(x_test)

In [24]:
from sklearn.metrics import confusion_matrix
cm_test = confusion_matrix(y_pred, y_test)

In [25]:
y_pred_train = classifier.predict(x_train)
cm_train = confusion_matrix(y_pred_train, y_train)

In [26]:
print()
print('Accuracy for training set for Random Forest = {}'.format((cm_train[0][0] + cm_train[1][1])/len(y_train)))
print('Accuracy for test set for Random Forest = {}'.format((cm_test[0][0] + cm_test[1][1])/len(y_test)))


Accuracy for training set for Random Forest = 0.8853754940711462
Accuracy for test set for Random Forest = 0.46


In [27]:
submission = pd.DataFrame({"ID":df_test["ID"],"Target":y_pred})
submission.to_csv("submission.csv", index=False)

In [28]:
submission

,ID,Target
0,128,0
1,232,1
2,220,1
3,184,0
4,25,1
5,56,0
6,28,1
7,125,1
8,139,0
9,67,0
